# Import dependencies

In [ ]:
import pandas as pd
import requests
import json

from helper_function.helper_map import plot_order_locations, get_map_by_vehicle
from helper_function.notebook_helpers import get_minutes_from_datetime,  show_vehicle_routes

from cuopt_thin_client import CuOptServiceClient

This grocery store is gaining popularity and people are placing orders. Instead of doing groceries themselves, they are getting a service like doordash so drivers can pick up their order from a selected grocery store and deliver it to their home address. 
Now, you're working as an Optimization Scientist in this company. You are given input about the customer's order, including the grocery store from which it must be picked up and their home address to which it must be delivered, their demand, depots, and the available fleet of vehicles. Again, you must calculate a route for each vehicle and ensure all orders are fulfilled while minimizing vehicles' travel time and cost. The same vehicle must handle both the pickup and delivery of the same order, and the pickup of the order must occur prior to the delivery. You can utilize cuOpt once again! All you need to do is read the input data and preprocess the data. Once all the data is ready, you save it to one dictionary and send it to cuOpt, which once again does all the hard computation for you. In this notebook, we will walk through the steps of this example.

# Read input data from CSV files
For the Pickup and Delivery (PDP) use case, we need 3 datasets with the following features:

- Depots
    - Name
    - Location
    - Start and end time (operation hours)
- Orders
    - Order name (includes 'pickup' or 'delivery')
    - Location
    - Start and end time (store hours for 'pickup', customer time windows for 'dropoff')
    - Demand (package weight and whether it is pickup or delivery)
    - Service time
- Vehicles
    - Name/ID Number
    - Vehicle type (car or bike)
    - Start and end time (vehicle/driver shift hours)
    - vehicle location
    - Capacity
    - Max distance
    
You may have additional features depending on the problem at hand.

In [ ]:
DATA_PATH = "data/"

orders_df = pd.read_csv(DATA_PATH+"orders_pdp.csv")
orders_df.rename(columns={"Latitude": "lat", "Longitude": "lng"}, inplace=True)
depot_df = pd.read_csv(DATA_PATH+"depots_pdp.csv")
depot_df.rename(columns={"Latitude": "lat", "Longitude": "lng"}, inplace=True)
vehicles_df = pd.read_csv(DATA_PATH+"vehicles_pdp.csv")

In [ ]:
n_depots = len(depot_df.index)
n_orders = len(orders_df.index)
n_vehicles = len(vehicles_df.index)

n_loc_total = n_orders + n_depots

In [ ]:
locations_df = pd.concat([orders_df, depot_df]).reset_index(drop=True)

# Create Cost Matrix

The **cost matrix** models the cost between each pair of locations.  It is used by cuOpt to compute the cost of traveling from any location to any other. The cost matrix needs to be a square matrix of dimension equal to the total number of locations which inlcludes both depots and orders. In this Vehicle Routing Problem, our cost metric is travel time. This is cost we want to minimize. 

To build a a cost matrix of live traffic data, we need to use a third party map data provider. In this workflow, the cost matrix will calculate the travel time in minutes between each two pairs of locations which we build using OSRM. 

In practical applications, you can integrate this to a third-party map data provider like Esri or Google Maps to get live traffic data and run dynamic/real-time re-routing using cuOpt.

In this PDP use case we have a mixed fleet consisting of two different types of vehicles: cars and bikes. We can imagine that these two different types of vehicles have different travel time in the crowded streets of NYC. A car is more likely to be stuck in traffic whereas a bike can bypass it. We create a different cost matrix for each vehicle type

In [ ]:
def build_car_cost_matrix(df):
    latitude = df.lat.to_numpy()
    longitude = df.lng.to_numpy()
    
    locations=""
    n_orders = len(df)
    for i in range(n_orders):
        locations = locations + "{},{};".format(longitude[i], latitude[i])
    r = requests.get("http://router.project-osrm.org/table/v1/car/"+ locations[:-1])
    routes = json.loads(r.content)
    
    # OSRM returns duration in seconds. Here we are converting to minutes
    for i in routes['durations']:
        i[:] = [x / 60 for x in i]
    
    coords_index = { i: (latitude[i], longitude[i]) for i in range(df.shape[0])}
    time_matrix = pd.DataFrame(routes['durations'])
    
    return time_matrix

In [ ]:
def build_bike_cost_matrix(df):
    latitude = df.lat.to_numpy()
    longitude = df.lng.to_numpy()
    
    locations=""
    n_orders = len(df)
    for i in range(n_orders):
        locations = locations + "{},{};".format(longitude[i], latitude[i])
    r = requests.get("http://router.project-osrm.org/table/v1/bike/"+ locations[:-1])
    routes = json.loads(r.content)
    
    # OSRM returns duration in seconds. Here we are converting to minutes
    for i in routes['durations']:
        i[:] = [x / 60 for x in i]
    
    coords_index = { i: (latitude[i], longitude[i]) for i in range(df.shape[0])}
    time_matrix = pd.DataFrame(routes['durations'])
    
    return time_matrix

In [ ]:
bike_cost_matrix_df = build_bike_cost_matrix(locations_df) 

In [ ]:
car_cost_matrix_df = build_car_cost_matrix(locations_df) 

# Visualize the locations

Before we use cuOpt to calculate the optimal routes, let's map all the locations.

In the map below, the depot is symbolized by the gray pin. The green pins are the pickup locations and the blue pins are delivery locations. 

In [ ]:
plot_order_locations(locations_df, locations_df[['lat', 'lng']], pdp=0)

## Set task data

Here we take our raw data from the csv file and convert it into data that we can send to the cuOpt solver.

- task_locations is the list of stores where tasks have placed an order.

- pickup_and_delivery_pairs is a list that maps a pickup order to its corresponding delivery order. 

- task_time_windows is the list of integer representation of opening hours for each store for 'pickup', and customer indicated time windows for 'dropoff'. We convert the UTC timestamp to epoch time (integer representation in minutes).

- service_times is the list of the length of time for orders to be picked up or dropped off once the vehicle reaches the location. Here, these values are between 15 and 30 minutes.

- demand is the list of weight demand for each order. Here, these values are between 15 and 40 pounds. 

In [ ]:
# Mapping pickups to deliveries

npair_orders = int(len(orders_df)/2)
pickup_locations = [i for i in range(npair_orders)]
delivery_locations = [i + npair_orders for i in range(npair_orders)]

pickup_and_delivery_pairs = list(zip(pickup_locations, delivery_locations))

In [ ]:
# Setting Order locations
all_task_locations = pickup_locations + delivery_locations

In [ ]:
# Earliest a delivery can be made
order_tw_earliest = orders_df['delivery_start'].apply(get_minutes_from_datetime)
# Latest a delivery can be made
order_tw_latest = orders_df['delivery_end'].apply(get_minutes_from_datetime)

task_time_windows = list(zip(order_tw_earliest, order_tw_latest)) 

# Service time required for the delivery/service
service_time = orders_df['service_time'].values.tolist()

In [ ]:
order_demand = [orders_df['order_wt'].values.tolist()]

## Set fleet data

Here we take our raw data from the csv file and convert it into data that we can send to the cuOpt solver.

- vehicle_locations is a list of the start and end location of the vehicles. In this example, we only have one depot so all the vehicles will start and end in the same location. 

- capacities is a list of how many orders each vehicle can carry in weight. In this example, a car can carry 800 lb and a bike can carry 50 lb.

- vehicle_time_windows is a list of the integer representation of the operating time of each vehicle. Equivalently, the shift of each vehicle driver. We convert the UTC timestamp to epoch time (integer representation in minutes).

- vehicle_types is a list that indicates whether each vehicle is a car or a bike. 


In [ ]:
vehicles_df['vehicle_start_in_minutes'] = vehicles_df['vehicle_start'].apply(get_minutes_from_datetime)
vehicles_df['vehicle_end_in_minutes'] = vehicles_df['vehicle_end'].apply(get_minutes_from_datetime)

# Earliest a vehicle can start 
v_tw_earliest = vehicles_df['vehicle_start_in_minutes']

# Latest a vehicle will be working
v_tw_latest = vehicles_df['vehicle_end_in_minutes']

vehicle_time_windows = list(zip(v_tw_earliest, v_tw_latest)) 

In [ ]:
vehicle_capacity = [vehicles_df['vehicle_capacity'].values.tolist()]

In [ ]:
vehicle_locations = list(zip(([len(orders_df)]*len(vehicles_df)), ([len(orders_df)]*len(vehicles_df))))

In [ ]:
vehicle_type_map = {"car":0, "bike":1}
vehicle_types = vehicles_df["vehicle_type"].replace(vehicle_type_map).tolist()

## Set solver configuration

Before we send our data to the cuOpt solver, we will add a few several configuration settings.

- the time_limit is the maximum time allotted to find a solution. This depends on the user, who has the flexibility of setting a higher time‑limit for better results. 

- the number of parallel agents (climbers) examining the solution search space.

If the user wants the first solution, then around 2-3 seconds for 2000-4000 climbers are enough. cuOpt solver does not interrupt the initial solution. So if the user specifies a shorter time than it takes for the initial solution, the initial solution is returned when it is computed. Increasing the number of climbers will increase the time it takes to compute the initial solution.
By default, the number of climbers is chosen by considering occupancy of a small GPU and experimented runtime vs number of climbers trade-off (that is, the best result in shortest time). Normally 1024 is a good start.

In [ ]:
# Set the time limit for solver to run
# Set number of climbers that will try to search for an optimal routes in parallel
time_limit = 0.1
number_of_climbers = 1024

## Save data in a dictionary

Here, we take all the data we have prepared so far and save it to one dictionary. This includes the cost matrices, task data, fleet data, and solver config. This is all the data that cuOpt needs to solve our PDP problem. 

In [ ]:
cuopt_problem_data = {
    "cost_matrix_data": {
        "cost_matrix": {
            0: car_cost_matrix_df.to_numpy().tolist(),
            1: bike_cost_matrix_df.to_numpy().tolist()
        }
    },
    "task_data": {
        "task_locations": all_task_locations,
        "pickup_and_delivery_pairs": pickup_and_delivery_pairs,
        "task_time_windows": task_time_windows,
        "service_times": service_time,
        "demand": order_demand,
    },
    "fleet_data": {
        "vehicle_locations": vehicle_locations,
        "capacities": vehicle_capacity,
        "vehicle_time_windows": vehicle_time_windows,
        "vehicle_types": vehicle_types,
    },
    "solver_config": {
        "time_limit": time_limit,
        "number_of_climbers": number_of_climbers
    }
}

## Create a Service Client Instance

Now that we have prepared all of our data, we can establish a connection to the cuOpt service. 

In the cell below, paste your client ID and Secret that you received via email in order to authenticate the connection. 

Here, we create an instance of the cuOpt Service Client to establish a connection. 

In [ ]:
cuopt_client_id = "<YOUR CLIENT ID>"
cuopt_client_secret = "<YOUR CLIENT SECRET>"

cuopt_service_client = CuOptServiceClient(
    client_id=cuopt_client_id,
    client_secret=cuopt_client_secret,
    )

## Send data to the cuOpt service and get the routes

When using the Python SDK or microservice, we need to send all the data in to cuOpt in multiple steps or API calls. 

When using the managed service, we send all the data in at once. The Python thin client includes an endpoint that send the data over to the service and returns the service output.

In [ ]:
# Solve the problem
solver_response = cuopt_service_client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]["solver_response"]

location_names = locations_df['name'].values.tolist()

if solver_resp["status"] == 0:
    print("Cost for the routing in distance: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    show_vehicle_routes(solver_resp, location_names)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

## Visualize the routes

In this example, not all vehicles are dispatched. It is possible that vehicle 0 is not dispatched but vehicle 1 is.  

In the drop down menu below, you can select different vehicle ID's to see if they are dispatched. If they are, we print their assigned route on a map. 

Generating a route and map uses third party tools and takes about 30 seconds to run. 

In [ ]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import interact

w = widgets.Dropdown(
    options = list(vehicles_df.index.values),
    description='Vehicle ID:',
)

def on_change(value):
    if str(value) in list(solver_resp['vehicle_data'].keys()):
        curr_route_df = pd.DataFrame(solver_resp["vehicle_data"][str(value)]['route'], columns=["stop_index"])
        curr_route_df = pd.merge(curr_route_df, locations_df, how="left", left_on=["stop_index"], right_on=[locations_df.index])
        display(get_map_by_vehicle(curr_route_df))        
    else:
        print("This Vehicle is not assigned to any order!!")

interact(on_change, value=w) 